># **FAQ ChatBOT**
># **This Notebook is created by: [mahdi khoshmaram](https://github.com/mahdi-khoshmaram)** 🤗
>
> In this notebook, the goal is to design a chatbot to map the User prompt to FAQ questions. For developing this chatbot I used:
>*   A Bert model **[sentence transformers](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2)** for **Sentence Embedding** which supports persian language.
>*  **ChromaDB**, an open-source vector database designed for the efficient storage and retrieval of vector embeddings, for handling my embeddings.
>*  **Resdis** an in-memory database for **Chatbot's RPS**(Request Per Second) handeling.
>* **[Hazm](https://www.roshan-ai.ir/hazm/docs/index.htmlhttps://www.roshan-ai.ir/hazm/docs/index.html)**, a python library based on NLTK to preprocess Persian text.


## ⭐ Like this notebook? Support by starring the repo!  

If you found this notebook helpful, consider giving a ⭐ to the repository! It helps and motivates me to create more useful content.  

[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](your-repo-link-here)  

Your support is greatly appreciated! 🚀

In [1]:
%%capture
%pip install chromadb
%pip install hazm
%pip install redis

In [166]:
from chromadb.utils import embedding_functions
from hazm import *
import chromadb
import shutil
import redis
import json
import time
import re

In [3]:
%%capture
!apt-get update
!apt-get install redis-server
!redis-server --daemonize yes

In [4]:
class Preprocess:
    def __init__(self, remove_stopwords=True, lemmatize=True):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()
        self.stopwords = set(stopwords_list())
        self.remove_stopwords = remove_stopwords
        self.lemmatize = lemmatize

    def clean(self, text):
        def process(t):
            t = self.normalizer.normalize(t)
            tokens = word_tokenize(t)

            if self.remove_stopwords:
                tokens = [word for word in tokens if word not in self.stopwords]
            if self.lemmatize:
                tokens = [self.lemmatizer.lemmatize(word).split("#")[0] for word in tokens]

            tokens = [re.sub(r"[^\w\s]", "", word) for word in tokens]
            tokens = [word for word in tokens if word.strip()]
            return " ".join(tokens)

        if isinstance(text, list):
            return [process(t) for t in text]
        return process(text)

In [294]:
class VectorMind:
    def __init__(self, modelName, chromaCollectionName, space, do_preprocess, db_path='./chromadb'):
        self.client = chromadb.PersistentClient(path=db_path)
        self.chromaCollectionName = chromaCollectionName
        self.preprocess = Preprocess()
        self.modelName = modelName
        self.space = space
        self.do_preprocess = do_preprocess

    def embedFaqAndSave(self, FaqQuestions):
        self.FaqQuestions = FaqQuestions
        self.model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = self.modelName)

        if self.chromaCollectionName in self.client.list_collections():
            self.client.delete_collection(name = self.chromaCollectionName)

        self.collection = self.client.create_collection(
            name = self.chromaCollectionName,
            embedding_function = self.model,
            metadata={
                "hnsw:space": self.space,
                "hnsw:search_ef": 100}
            )


        if self.do_preprocess:
            self.FaqQuestionsForDB = self.preprocess.clean(self.FaqQuestions)
            self.faqIDs = [str(index) for index in range(len(self.FaqQuestionsForDB))]
        else:
            self.FaqQuestionsForDB = self.FaqQuestions
            self.faqIDs = [str(index) for index in range(len(self.FaqQuestionsForDB))]

        self.collection.add(
            documents = list(self.FaqQuestionsForDB),
            ids = list(self.faqIDs)
            )


    def query(self, prompt, top_k=2, threshold=50):

        if self.do_preprocess:
            self.Prompt = self.preprocess.clean(prompt)
        else:
            self.Prompt = prompt
        if isinstance(self.Prompt, str):
            self.Prompt = [self.Prompt]

        self.top_k = top_k
        self.numberOfPrompts = len(self.Prompt)

        self.query_result = self.collection.query(
            query_texts=self.Prompt,
            n_results = self.top_k)

        result_ids = self.query_result["ids"]
        result_documents = self.query_result["documents"]
        result_distances = self.query_result["distances"]
        result_similarity = [[round((1-distance)*100,2) for distance in row] for row in result_distances]

        # One of longest dictionary comprehensions I've ever wrote! =)))))))
        self.result = {f"Prompt{i}":{"UserPrompt":self.Prompt[i], "results":{f"result{k}":{"FAQ":result_documents[i][k], "Index": result_ids[i][k], "similarity":result_similarity[i][k]} for k in range(self.top_k) if result_similarity[i][k]>=threshold}} for i in range(self.numberOfPrompts)}
        self.result = json.dumps(dict(self.result), ensure_ascii=False, indent=4)

        return self.result

In [295]:
class Chatbot(VectorMind):
    def __init__(self, faqQuestions, modelName, chromaCollectionName, space, do_preprocess, db_path='./chromadb',host='localhost', port=6379, db=0, rate_limit=1):
        super().__init__(modelName, chromaCollectionName, space, do_preprocess, db_path)
        super().embedFaqAndSave(FaqQuestions=faqQuestions)
        self.redis_client = redis.Redis(host=host, port=port, db=db)
        self.rate_limit = rate_limit

    def is_allowed(self, user_id=1):
        key = f"rate_limit:{user_id}"
        last_request_time = self.redis_client.get(key)
        current_time = time.time()

        if last_request_time and (current_time - float(last_request_time) < self.rate_limit):
            return False
        self.redis_client.set(key, current_time)
        return True

    def call(self, myPrompt, top_k=1, threshold=50, user_id=1):
        if not self.is_allowed(user_id):
            return "Too many requests! Try again in a second."

        response = super().query(myPrompt, top_k=top_k, threshold=threshold)
        return response


In [296]:
faq_questions = [
    "در صورت فراموشی رمز موبایل بانک و یا اینترنت بانک چه اقدامی بایستی انجام داد؟",
    "مدت زمان دریافت کارت فرارفاه چقدر است؟",
    "سقف انتقال وجه در برنامه موبایل بانک و اینترنت بانک چقدر است؟",
    "برای انتقال وجه پایا و ساتنا باید از چه رمزی استفاده کرد؟",
    "برای ثبت چک صیادی از چه نوع رمزی باید استفاده کرد؟",
    "آیا دریافت تسهیلات بدون ضامن و بدون سپرده گذاری درحال حاضر ممکن است؟",
    "علت عدم ورود به موبایل بانک درگوشی شیائومی چیست؟",
    "تغییر رمز اول و دوم موبایل بانک از چه روش هایی انجام می شود؟",
    "رمز پویا از چه طریقی فعال می شود؟",
    "در اینترنت بانک حقوقی چگونه می شود تایید نهایی را انجام داد؟",
    "آیا امکان دریافت فایل اکسل از طریق موبایل بانک وجود دارد؟",
    "به چه طریقی می توان شماره شبا و حساب را از بانک رفاه دریافت نمود؟",
    "نحوه تایید عملیات انجام تراکنش در موبایل بانک و اینترنت بانک چگونه است؟",
    "قانون جدید صدور چک"
]

In [297]:
chatbot = Chatbot(
    faqQuestions=faq_questions,
    modelName="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    chromaCollectionName="faqCollection",
    space="cosine",
    do_preprocess=False)

In [300]:
myPrompts = ["چطوری رمز پویا بگیرم؟", "چطوری شماره شبا بگیرم؟", "سلام، من مهدی ام!"]
print(chatbot.call(myPrompts, top_k=1, threshold=50))

{
    "Prompt0": {
        "UserPrompt": "چطوری رمز پویا بگیرم؟",
        "results": {
            "result0": {
                "FAQ": "رمز پویا از چه طریقی فعال می شود؟",
                "Index": "8",
                "similarity": 72.33
            }
        }
    },
    "Prompt1": {
        "UserPrompt": "چطوری شماره شبا بگیرم؟",
        "results": {
            "result0": {
                "FAQ": "به چه طریقی می توان شماره شبا و حساب را از بانک رفاه دریافت نمود؟",
                "Index": "11",
                "similarity": 50.5
            }
        }
    },
    "Prompt2": {
        "UserPrompt": "سلام، من مهدی ام!",
        "results": {}
    }
}


# **Test Cases**

# **Test Case 1:**

*   `top_k = 1`:   returns top 1 similar FAQ question


*   `threshold = 10`: returns similar FAQ questions with similarity rate higher than 10%

In [306]:
myPrompts = ["چطوری رمز پویا بگیرم؟", "چطوری شماره شبا بگیرم؟", "سلام، من مهدی ام!"]
for prompt in myPrompts:
    print(chatbot.call(prompt, top_k=2, threshold=50))
    time.sleep(0.5)

{
    "Prompt0": {
        "UserPrompt": "چطوری رمز پویا بگیرم؟",
        "results": {
            "result0": {
                "FAQ": "رمز پویا از چه طریقی فعال می شود؟",
                "Index": "8",
                "similarity": 72.33
            }
        }
    }
}
Too many requests! Try again in a second.
{
    "Prompt0": {
        "UserPrompt": "سلام، من مهدی ام!",
        "results": {}
    }
}


# **Test Case 2:**

*   `top_k = 2`:   returns top 2 similar FAQ question


*   `threshold = 25`: returns similar FAQ questions with similarity rate higher than 25%

In [11]:
myPrompts = ["چطوری رمز پویا بگیرم؟", "چطوری شماره شبا بگیرم؟", "سلام، من مهدی ام!"]
for prompt in myPrompts:
    print(chatbot.call(prompt, top_k=2, threshold=25))
    time.sleep(1)

[[('UserPrompt:چطوری رمز پویا بگیرم؟', 'FAQ:رمز پویا از چه طریقی فعال می شود؟', 'IndexOfFAQ = 8', 'Similarity = 72.33%'), ('UserPrompt:چطوری رمز پویا بگیرم؟', 'FAQ:برای انتقال وجه پایا و ساتنا باید از چه رمزی استفاده کرد؟', 'IndexOfFAQ = 3', 'Similarity = 39.92%')]]
[[('UserPrompt:چطوری شماره شبا بگیرم؟', 'FAQ:به چه طریقی می توان شماره شبا و حساب را از بانک رفاه دریافت نمود؟', 'IndexOfFAQ = 11', 'Similarity = 50.5%'), ('UserPrompt:چطوری شماره شبا بگیرم؟', 'FAQ:مدت زمان دریافت کارت فرارفاه چقدر است؟', 'IndexOfFAQ = 1', 'Similarity = 37.75%')]]
[[]]


# **Test Case 3:**

*   `top_k = 2`:   returns top 2 similar FAQ question


*   `threshold = 50`: returns similar FAQ questions with similarity rate higher than 50%

In [51]:
myPrompts = ["چطوری رمز پویا بگیرم؟", "چطوری شماره شبا بگیرم؟", "سلام، من مهدی ام!"]
for prompt in myPrompts:
    print(chatbot.call(prompt, top_k=2, threshold=50))
    time.sleep(1)

[[('UserPrompt:چطوری رمز پویا بگیرم؟', 'FAQ:رمز پویا از چه طریقی فعال می شود؟', 'IndexOfFAQ = 8', 'Similarity = 72.33%')]]
[[('UserPrompt:چطوری شماره شبا بگیرم؟', 'FAQ:به چه طریقی می توان شماره شبا و حساب را از بانک رفاه دریافت نمود؟', 'IndexOfFAQ = 11', 'Similarity = 50.5%')]]
[[]]
